# ResNet ARCHITECTURE

This notebook implement the ResNet architecture for image recognition using CNN.
You can find the paper describing this architecture by following the link:
https://arxiv.org/pdf/1512.03385.pdf

**Here is the global architecture of a ResNet network : **
<img src="images/resnet.png" style="width:572px;height:1314px;">

In [ ]:
import numpy as np
import math
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.python.framework import ops

In [ ]:
%run Utilities.ipynb

In [ ]:
trainSet = loadFile('images.npy')
labelTrainSet = loadFile('labels.npy')

In [ ]:
showImage(inTainSet= trainSet, atPosition=1999)

In [ ]:
normalize_image(trainingSet=trainSet)

In [ ]:
unique_array = np.unique(labelTrainSet) #take unique values from labelTrainSet (increasing ordering)

for i in range(0,len(labelTrainSet)):   #pass through all array
    labelTrainSet[i] = np.argwhere(unique_array==labelTrainSet[i])[0][0] #replace the current value by the index of the value in the unique_array
    
reshapeLabels = labelTrainSet.reshape(labelTrainSet.shape[0], 1) #reshape in shape (m,1)

labelSet = convert_to_one_hot(reshapeLabels.T.astype(np.int32), len(unique_array)).T #conversion in one hot with shape (m, n_c)
                

In [ ]:
s = np.arange(trainSet.shape[0])
np.random.shuffle(s)
trainSet = trainSet[s]
labelSet = labelSet[s]

In [ ]:
X_train = trainSet[:-200,:,:,:]
Y_tain = labelSet[:-200,:]

X_test = trainSet[-200:,:,:,:]
Y_test = labelSet[-200:,:]

In [ ]:
def create_activation_layer(input):  
    ## "All hidden layers are equipped with the rectification non-linearity"
    layer = tf.nn.relu(input)
 
    return layer

In [ ]:
def create_pooling_layer_gad(input, parameters):  

    ksize = [1,input.shape[1],input.shape[2],1]
    strides = parameters['strides_pooling'] #example [1, 2, 2, 1] corresponding [batch_size, img_size, img_size, number of channel]
    padding = parameters['padding'] #example 'SAME' or 'VALID'
    
    ## "Max-pooling is performed over a 2x2 pixel window, with stride 2".  
    layer = tf.nn.avg_pool(input, ksize= ksize, strides= strides, padding= padding)
 
    return layer

# Create Residual layer

**Here is the architecture of a ResNet unit block : **
<img src="images/resnet_unit_block.png" style="width:408px;height:244px;">

In [ ]:
def create_residual_layer(input, parameters):
    
    conv_filter_size = parameters['filter_size_convRes']
    num_filters = parameters['num_filters_convRes']
    padding = parameters['padding_convRes']
    strides = parameters['strides_convRes']
    layer_number = parameters['layer_number']
    dottedShortcut = parameters['dottedShortcut']
    
    num_input_channels1 = input.shape[3]
    
    conv_layer1 = create_convolutional_layer(input,
               num_input_channels1, 
               conv_filter_size,        
               num_filters,
               padding,
               strides,
               layer_number)
    
    bn_layer1 = tf.layers.batch_normalization(conv_layer1)
    
    act_layer = create_activation_layer(bn_layer1)
    
    num_input_channels2 = act_layer.shape[3]
    
    conv_layer2 = create_convolutional_layer(act_layer,
               num_input_channels2, 
               conv_filter_size,        
               num_filters,
               padding,
               [1,1,1,1],
               layer_number + '_second')
    
    bn_layer2 = tf.layers.batch_normalization(conv_layer2)
    
    if strides == [1,1,1,1]:
        res_layer = tf.math.add(bn_layer2, input)
    elif dottedShortcut == 'A':
        res_layer = tf.math.add(tf.image.resize_image_with_pad(bn_layer2, target_height= int(input.shape[1]), target_width= int(input.shape[2])), input)
    elif dottedShortcut == 'B':
        res = create_convolutional_layer(input, input.shape[3], 1, bn_layer2.shape[3], 'SAME', [1,2,2,1], layer_number + 'B', use_bias = False)   
        res_layer = tf.math.add(bn_layer2, res)
    else:
        res_layer = bn_layer2
            
    output_layer = create_activation_layer(res_layer)
    
    return output_layer

# Create FC layer

In [ ]:
def create_fully_connected_layer(input, num_input_channels, num_output_channel, layer_number):
        
    ## We shall define the weights that will be trained using create_weights function.
    weights = create_weights(name='Wfc_'+layer_number, shape=[input.shape[1], input.shape[2], num_input_channels, num_output_channel])
    ## We create biases using the create_biases function. These are also trained.
    biases = create_biases(name='Bfc_'+layer_number, size=num_output_channel)
    
    layer = tf.nn.conv2d(input=input,
                     filter=weights,
                     strides=[1, 1, 1, 1],
                    padding= 'VALID')
    
    layer += layer + biases
    
    return layer

# Create softmax layer

In [ ]:
def create_softmax_layer(networkOutput, Y):
    """
    Computes the cost
    
    Arguments:
    Output -- output of forward propagation (output of the last LINEAR unit), of shape (6, number of examples)
    Y -- "true" labels vector placeholder, same shape as output
    
    Returns:
    the coss entropy
    """
    #print(Y.shape)
    #print(networkOutput.shape)
    return tf.nn.softmax_cross_entropy_with_logits_v2(logits= networkOutput, labels= Y) 

# Compute cost

In [ ]:
def compute_cost(softmaxOutput):
    """
    Computes the cost
    
    Arguments:
    softmaxOutput -- output of the softmax layer
    
    Returns:
    cost - Tensor of the cost function
    """
    
    cost = tf.reduce_mean(softmaxOutput)
    
    return cost

# Create architectures

In [ ]:
def get_parameters_configurationTest():
    filter_size_convRes1 = 3
    num_filters_convRes1 = 64
    strides_convRes1 = [1, 1, 1, 1]
    padding_convRes1 = 'SAME'
    dottedShortcut1 = 'B'
    layer_number_convRes1 = '1'
    
    filter_size_convRes2 = 3
    num_filters_convRes2 = 64
    strides_convRes2 = [1, 1, 1, 1]
    padding_convRes2 = 'SAME'
    dottedShortcut2 = 'B'
    layer_number_convRes2 = '2'
    
    filter_size_convRes3 = 3
    num_filters_convRes3 = 64
    strides_convRes3 = [1, 1, 1, 1]
    padding_convRes3 = 'SAME'
    dottedShortcut3 = 'B'
    layer_number_convRes3 = '3'
    
    filter_size_convRes4 = 3
    num_filters_convRes4 = 128
    strides_convRes4 = [1, 2, 2, 1]
    padding_convRes4 = 'SAME'
    dottedShortcut4 = 'B'
    layer_number_convRes4 = '4'
    
    filter_size_convRes5 = 3
    num_filters_convRes5 = 128
    strides_convRes5 = [1, 1, 1, 1]
    padding_convRes5 = 'SAME'
    dottedShortcut5 = 'B'
    layer_number_convRes5 = '5'
    
    filter_size_convRes6 = 3
    num_filters_convRes6 = 128
    strides_convRes6 = [1, 1, 1, 1]
    padding_convRes6 = 'SAME'
    dottedShortcut6 = 'B'
    layer_number_convRes6 = '6'
    
    filter_size_convRes7 = 3
    num_filters_convRes7 = 128
    strides_convRes7 = [1, 1, 1, 1]
    padding_convRes7 = 'SAME'
    dottedShortcut7 = 'B'
    layer_number_convRes7 = '7'
    
    filter_size_convRes8 = 3
    num_filters_convRes8 = 256
    strides_convRes8 = [1, 2, 2, 1]
    padding_convRes8 = 'SAME'
    dottedShortcut8 = 'B'
    layer_number_convRes8 = '8'
    
    filter_size_convRes9 = 3
    num_filters_convRes9 = 256
    strides_convRes9 = [1, 1, 1, 1]
    padding_convRes9 = 'SAME'
    dottedShortcut9 = 'B'
    layer_number_convRes9 = '9'
    
    filter_size_convRes10 = 3
    num_filters_convRes10 = 256
    strides_convRes10 = [1, 1, 1, 1]
    padding_convRes10 = 'SAME'
    dottedShortcut10 = 'B'
    layer_number_convRes10 = '10'
    
    filter_size_convRes11 = 3
    num_filters_convRes11 = 256
    strides_convRes11 = [1, 1, 1, 1]
    padding_convRes11 = 'SAME'
    dottedShortcut11 = 'B'
    layer_number_convRes11 = '11'
    
    filter_size_convRes12 = 3
    num_filters_convRes12 = 256
    strides_convRes12 = [1, 1, 1, 1]
    padding_convRes12 = 'SAME'
    dottedShortcut12 = 'B'
    layer_number_convRes12 = '12'
    
    filter_size_convRes13 = 3
    num_filters_convRes13 = 256
    strides_convRes13 = [1, 1, 1, 1]
    padding_convRes13 = 'SAME'
    dottedShortcut13 = 'B'
    layer_number_convRes13 = '13'
    
    filter_size_convRes14 = 3
    num_filters_convRes14 = 512
    strides_convRes14 = [1, 2, 2, 1]
    padding_convRes14 = 'SAME'
    dottedShortcut14 = 'B'
    layer_number_convRes14 = '14'
    
    filter_size_convRes15 = 3
    num_filters_convRes15 = 512
    strides_convRes15 = [1, 1, 1, 1]
    padding_convRes15 = 'SAME'
    dottedShortcut15 = 'B'
    layer_number_convRes15 = '15'
    
    filter_size_convRes16 = 3
    num_filters_convRes16 = 512
    strides_convRes16 = [1, 1, 1, 1]
    padding_convRes16 = 'SAME'
    dottedShortcut16 = 'B'
    layer_number_convRes16 = '16'
    
    parameters_conv1 = {
        
    }
    
    parameters_res1 = {
                            'filter_size_convRes': filter_size_convRes1,
                            'num_filters_convRes': num_filters_convRes1,
                            'strides_convRes': strides_convRes1,
                            'padding_convRes': padding_convRes1,
                            'layer_number': layer_number_convRes1,
                            'dottedShortcut': dottedShortcut1
                            }
    
    parameters_res2 = {
                            'filter_size_convRes': filter_size_convRes2,
                            'num_filters_convRes': num_filters_convRes2,
                            'strides_convRes': strides_convRes2,
                            'padding_convRes': padding_convRes2,
                            'layer_number': layer_number_convRes2,
                            'dottedShortcut': dottedShortcut2
                            }
    
    parameters_res3 = {
                            'filter_size_convRes': filter_size_convRes3,
                            'num_filters_convRes': num_filters_convRes3,
                            'strides_convRes': strides_convRes3,
                            'padding_convRes': padding_convRes3,
                            'layer_number': layer_number_convRes3,
                            'dottedShortcut': dottedShortcut3
                            }
    
    parameters_res4 = {
                            'filter_size_convRes': filter_size_convRes4,
                            'num_filters_convRes': num_filters_convRes4,
                            'strides_convRes': strides_convRes4,
                            'padding_convRes': padding_convRes4,
                            'layer_number': layer_number_convRes4,
                            'dottedShortcut': dottedShortcut4
                            }
    
    parameters_res5 = {
                            'filter_size_convRes': filter_size_convRes5,
                            'num_filters_convRes': num_filters_convRes5,
                            'strides_convRes': strides_convRes5,
                            'padding_convRes': padding_convRes5,
                            'layer_number': layer_number_convRes5,
                            'dottedShortcut': dottedShortcut5
                            }
    
    parameters_res6 = {
                            'filter_size_convRes': filter_size_convRes6,
                            'num_filters_convRes': num_filters_convRes6,
                            'strides_convRes': strides_convRes6,
                            'padding_convRes': padding_convRes6,
                            'layer_number': layer_number_convRes6,
                            'dottedShortcut': dottedShortcut6
                            }
    
    parameters_res7 = {
                            'filter_size_convRes': filter_size_convRes7,
                            'num_filters_convRes': num_filters_convRes7,
                            'strides_convRes': strides_convRes7,
                            'padding_convRes': padding_convRes7,
                            'layer_number': layer_number_convRes7,
                            'dottedShortcut': dottedShortcut7
                            }
    
    parameters_res8 = {
                            'filter_size_convRes': filter_size_convRes8,
                            'num_filters_convRes': num_filters_convRes8,
                            'strides_convRes': strides_convRes8,
                            'padding_convRes': padding_convRes8,
                            'layer_number': layer_number_convRes8,
                            'dottedShortcut': dottedShortcut8
                            }
    
    parameters_res9 = {
                            'filter_size_convRes': filter_size_convRes9,
                            'num_filters_convRes': num_filters_convRes9,
                            'strides_convRes': strides_convRes9,
                            'padding_convRes': padding_convRes9,
                            'layer_number': layer_number_convRes9,
                            'dottedShortcut': dottedShortcut9
                            }
    
    parameters_res10 = {
                            'filter_size_convRes': filter_size_convRes10,
                            'num_filters_convRes': num_filters_convRes10,
                            'strides_convRes': strides_convRes10,
                            'padding_convRes': padding_convRes10,
                            'layer_number': layer_number_convRes10,
                            'dottedShortcut': dottedShortcut10
                            }
    
    parameters_res11 = {
                            'filter_size_convRes': filter_size_convRes11,
                            'num_filters_convRes': num_filters_convRes11,
                            'strides_convRes': strides_convRes11,
                            'padding_convRes': padding_convRes11,
                            'layer_number': layer_number_convRes11,
                            'dottedShortcut': dottedShortcut11
                            }
    
    parameters_res12 = {
                            'filter_size_convRes': filter_size_convRes12,
                            'num_filters_convRes': num_filters_convRes12,
                            'strides_convRes': strides_convRes12,
                            'padding_convRes': padding_convRes12,
                            'layer_number': layer_number_convRes12,
                            'dottedShortcut': dottedShortcut12
                            }
    
    parameters_res13 = {
                            'filter_size_convRes': filter_size_convRes13,
                            'num_filters_convRes': num_filters_convRes13,
                            'strides_convRes': strides_convRes13,
                            'padding_convRes': padding_convRes13,
                            'layer_number': layer_number_convRes13,
                            'dottedShortcut': dottedShortcut13
                            }
    
    parameters_res14 = {
                            'filter_size_convRes': filter_size_convRes14,
                            'num_filters_convRes': num_filters_convRes14,
                            'strides_convRes': strides_convRes14,
                            'padding_convRes': padding_convRes14,
                            'layer_number': layer_number_convRes14,
                            'dottedShortcut': dottedShortcut14
                            }
    
    parameters_res15 = {
                            'filter_size_convRes': filter_size_convRes15,
                            'num_filters_convRes': num_filters_convRes15,
                            'strides_convRes': strides_convRes15,
                            'padding_convRes': padding_convRes15,
                            'layer_number': layer_number_convRes15,
                            'dottedShortcut': dottedShortcut15
                            }
    
    parameters_res16 = {
                            'filter_size_convRes': filter_size_convRes16,
                            'num_filters_convRes': num_filters_convRes16,
                            'strides_convRes': strides_convRes16,
                            'padding_convRes': padding_convRes16,
                            'layer_number': layer_number_convRes16,
                            'dottedShortcut': dottedShortcut16
                            }
    
    parameters_pooling = {
                            'padding': 'VALID',
                            'strides_pooling': [1,1,1,1]
    }
    
    num_output_class = 4
    
    architecture= {
               'parameters_res1': parameters_res1,
               'parameters_res2': parameters_res2,
               'parameters_res3': parameters_res3,
               'parameters_res4': parameters_res4,
               'parameters_res5': parameters_res5,
               'parameters_res6': parameters_res6,
               'parameters_res7': parameters_res7,
               'parameters_res8': parameters_res8,
               'parameters_res9': parameters_res9,
               'parameters_res10': parameters_res10,
               'parameters_res11': parameters_res11,
               'parameters_res12': parameters_res12,
               'parameters_res13': parameters_res13,
               'parameters_res14': parameters_res14,
               'parameters_res15': parameters_res15,
               'parameters_res16': parameters_res16,
               'parameters_pooling': parameters_pooling,
               'num_output_class': num_output_class
    }
    
    return architecture

In [ ]:
def configuration_Test(X, architecture):
    
    parameters = {}
    
    parameters_res1 = architecture['parameters_res1']
    parameters_res2 = architecture['parameters_res2']
    parameters_res3 = architecture['parameters_res3']
    parameters_res4 = architecture['parameters_res4']
    parameters_res5 = architecture['parameters_res5']
    parameters_res6 = architecture['parameters_res6']
    parameters_res7 = architecture['parameters_res7']
    parameters_res8 = architecture['parameters_res8']
    parameters_res9 = architecture['parameters_res9']
    parameters_res10 = architecture['parameters_res10']
    parameters_res11 = architecture['parameters_res11']
    parameters_res12 = architecture['parameters_res12']
    parameters_res13 = architecture['parameters_res13']
    parameters_res14 = architecture['parameters_res14']
    parameters_res15 = architecture['parameters_res15']
    parameters_res16 = architecture['parameters_res16']
    
    parameters_pooling = architecture['parameters_pooling']

    num_output_class = architecture['num_output_class']
    
    size_pooling1 = [1, 2, 2, 1] 
    strides_pooling1 = [1, 2, 2, 1]
    padding1 = 'SAME'

    parameters_pooling1 = {
                            'size_pooling': size_pooling1,
                            'strides_pooling': strides_pooling1,
                            'padding': padding1
                            }
    
    layer_conv1 = create_convolutional_layer(input= X,conv_filter_size=9, num_input_channels= X.shape[3], num_filters= 64,padding= 'VALID',strides= [1,1,1,1], layer_number= 'conv1')
    
    layer_bn1 = tf.layers.batch_normalization(layer_conv1)
    layer_act1 = create_activation_layer(layer_bn1)
    
    layer_convRes1 = create_residual_layer(layer_act1, parameters_res1)
    layer_convRes2 = create_residual_layer(layer_convRes1, parameters_res2)
    layer_convRes3 = create_residual_layer(layer_convRes2, parameters_res3)
    
    layer_convRes4 = create_residual_layer(layer_convRes3, parameters_res4)
    layer_convRes5 = create_residual_layer(layer_convRes4, parameters_res5)
    layer_convRes6 = create_residual_layer(layer_convRes5, parameters_res6)
    layer_convRes7 = create_residual_layer(layer_convRes6, parameters_res7)
    
    layer_convRes8 = create_residual_layer(layer_convRes7, parameters_res8)
    layer_convRes9 = create_residual_layer(layer_convRes8, parameters_res9)
    layer_convRes10 = create_residual_layer(layer_convRes9, parameters_res10)
    layer_convRes11 = create_residual_layer(layer_convRes10, parameters_res11)
    layer_convRes12 = create_residual_layer(layer_convRes11, parameters_res12)
    layer_convRes13 = create_residual_layer(layer_convRes12, parameters_res13)
    
    layer_convRes14 = create_residual_layer(layer_convRes13, parameters_res14)
    layer_convRes15 = create_residual_layer(layer_convRes14, parameters_res15)
    layer_convRes16 = create_residual_layer(layer_convRes15, parameters_res16)
    
    layer_gad = create_pooling_layer_gad(layer_convRes16, parameters_pooling) #Global Average Pooling

    #layer_fc1 = create_fully_connected_layer(input=layer_gad, num_input_channels=layer_gad.shape[3], num_output_channel=num_output_class, layer_number='fc1')
    #layer_maxpool1 = create_pooling_layer(input=layer_convRes4, parameters=parameters_pooling1)
    # FLATTEN
    P2 = tf.contrib.layers.flatten(layer_gad)
    # FULLY-CONNECTED without non-linear activation function (not not call softmax).
    # 6 neurons in output layer. Hint: one of the arguments should be "activation_fn=None" 
    layer_fc1 = tf.contrib.layers.fully_connected(P2, num_outputs=4, activation_fn=None)
    ### END CODE HERE ###
    
    return layer_fc1, parameters

# Model routine

In [ ]:
def model(X_train, Y_train, X_test, Y_test, learning_rate = 0.01, configuration= 'configuration_A',
          num_epochs = 100, minibatch_size = 32, momentum = 0.9, use_nesterov = False, print_cost = True, beta = 0.001):
    """
    Implements a three-layer tensorflow neural network: LINEAR->RELU->LINEAR->RELU->LINEAR->SOFTMAX.
    
    Arguments:
    X_train -- training set, of shape (input size = 12288, number of training examples = 1080)
    Y_train -- test set, of shape (output size = 6, number of training examples = 1080)
    X_test -- training set, of shape (input size = 12288, number of training examples = 120)
    Y_test -- test set, of shape (output size = 6, number of test examples = 120)
    learning_rate -- learning rate of the optimization
    num_epochs -- number of epochs of the optimization loop
    minibatch_size -- size of a minibatch
    print_cost -- True to print the cost every 100 epochs
    
    Returns:
    parameters -- parameters learnt by the model. They can then be used to predict.
    """
    
    ops.reset_default_graph()                         # to be able to rerun the model without overwriting tf variables
    (m, n_w, n_h, n_c) = X_train.shape                # (m : number of examples in the train set, n_w: image width, n_h: image height, n_c: number of image channel)
    n_y = Y_train.shape[1]                            # n_y : output size
    costs = []                                        # To keep track of the cost
    
    # Create Placeholders
    X, Y = create_placeholders(img_size=n_w, num_channels=n_c, num_classes=n_y)
    
    # Forward propagation
    if configuration == 'configuration_Test':
        architecture = get_parameters_configurationTest()
        configuration_output, parameters = configuration_Test(X=X,architecture=architecture)
            
    #Create softmax layer
    softmax_layer = create_softmax_layer(Y=Y, networkOutput=configuration_output)
    
    # Cost function
    cost = compute_cost(softmaxOutput=softmax_layer)
    
    # Backpropagation: Define the tensorflow optimizer.
    optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum=momentum, use_nesterov=use_nesterov).minimize(cost)
    #gvs = optimizer.compute_gradients(cost)
    #capped_gvs = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gvs]
    #train_op = optimizer.apply_gradients(capped_gvs)
    
    # Initialize variables
    init = tf.global_variables_initializer()

    # Start the session
    with tf.Session() as sess:
        
        # Run the initialization
        sess.run(init)
        
        # Do the training loop
        for epoch in range(num_epochs):

            epoch_cost = 0.                       # Defines a cost related to an epoch
            num_minibatches = int(m / minibatch_size) # number of minibatches of size minibatch_size in the train set

            minibatches = random_mini_batches(X_train, Y_train, minibatch_size)

            for minibatch in minibatches:

                # Select a minibatch
                (minibatch_X, minibatch_Y) = minibatch

                # Run the session: execute "optimizer" and "cost"
                _, minibatch_cost = sess.run([optimizer, cost], feed_dict={X: minibatch_X, Y: minibatch_Y})
               
                epoch_cost += minibatch_cost / num_minibatches

                #print(minibatch_cost)
            # Print the cost every epoch
            if print_cost == True and epoch % 1 == 0:
                print ("Cost after epoch %i: %f" % (epoch, epoch_cost))
            if print_cost == True and epoch % 1 == 0:
                costs.append(epoch_cost)
                
        # plot the cost
        plt.plot(np.squeeze(costs))
        plt.ylabel('cost')
        plt.xlabel('iterations (per tens)')
        plt.title("Learning rate =" + str(learning_rate))
        plt.show()

        # Calculate the correct predictions
        predict_op = tf.reshape(tf.argmax(configuration_output, -1), [-1])
        true_label = tf.argmax(Y, 1)
        correct_prediction = tf.equal(predict_op, true_label)
        
        # Calculate accuracy on the test set
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
        train_accuracy = accuracy.eval({X: X_train, Y: Y_train})
        test_accuracy = accuracy.eval({X: X_test, Y: Y_test})
        print("Train Accuracy:", train_accuracy)
        print("Test Accuracy:", test_accuracy)
        
        return train_accuracy

In [ ]:
model(X_train=X_train, Y_train=Y_tain, X_test=X_test, Y_test=Y_test, configuration='configuration_Test', momentum= 0.9, num_epochs=50,learning_rate=0.01, minibatch_size=128, use_nesterov=False, print_cost= True)
